In [1]:
import os
from tqdm import *
import seaborn as sns
import pandas as pd
import pickle
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import lightgbm as lgb
import sklearn
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.metrics import mean_squared_error, f1_score, recall_score, precision_score

c:\users\pinlan\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
print(lgb.__version__)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)

2.1.1
1.16.2
0.23.0
0.19.1


In [5]:
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
 
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r

def get_path(path):
    file_name = []
    for i in os.listdir(path):
        c = i.split('_')[0]
        file_name.append(c)
    file_name = list(set(file_name))
    return file_name

def lgb_recall_score(y_true, y_hat,t):
    y_hat[y_hat>=t] = 1
    y_hat[y_hat<t] = 0
    return recall_score(y_true, y_hat, average='binary')

In [6]:
train_po_path = '../data/Motor_tain/Positive/'
train_ne_path = '../data/Motor_tain/Negative/'
train_po_file = get_path(train_po_path)
train_ne_file = get_path(train_ne_path)
test_path = '../data/Motor_testP/'
test_file_name = get_path(test_path)

In [7]:
def get_data(file_name,file_path):
    fe = defaultdict(list)
    for i in tqdm(file_name):
        data_b = pd.read_csv(file_path + i + '_B.csv')
        data_f = pd.read_csv(file_path + i +  '_F.csv')
        data_b['p'] = np.sqrt(data_b.ai1**2+data_b.ai2**2)
        data_f['p'] = np.sqrt(data_f.ai1**2+data_f.ai2**2)
        fe['idx'].append(i)

        fe['ai1_max_p'].append(data_b.p.max())
        fe['ai1_min_p'].append(data_b.p.min())
        fe['ai1_mean_p'].append(data_b.p.mean())
        fe['ai1_abs_mean_p'].append(data_b.p.abs().mean())
        fe['ai1_std_p'].append(data_b.p.std())
        fe['ai1_abs_skew_p'].append(data_b.p.abs().skew())
        fe['ai1_kurt_p'].append(data_b.p.kurt())
        fe['ai1_rms_p'].append(np.sqrt(np.mean(data_b.p**2)))
        
        fe['circle_ratio_b'].append(data_b[data_b['p']<0.04].shape[0])
        fe['ai1_0_b'].append(data_b[data_b['ai1']>0].shape[0])
        fe['ai1_max_b'].append(data_b.ai1.max())
        fe['ai1_min_b'].append(data_b.ai1.min())
        fe['ai1_mean_b'].append(data_b.ai1.mean())
        fe['ai1_abs_mean_b'].append(data_b.ai1.abs().mean())
        fe['ai1_std_b'].append(data_b.ai1.std())
        fe['ai1_abs_skew_b'].append(data_b.ai1.abs().skew())
        fe['ai1_kurt_b'].append(data_b.ai1.kurt())
        fe['ai1_rms_b'].append(np.sqrt(np.mean(data_b.ai1**2)))
        
        fe['ai2_max_b'].append(data_b.ai2.max())
        fe['ai2_min_b'].append(data_b.ai2.min())
        fe['ai2_mean_b'].append(data_b.ai2.mean())
        fe['ai2_abs_mean_b'].append(data_b.ai2.abs().mean())
        fe['ai2_std_b'].append(data_b.ai2.std())
        fe['ai2_abs_skew_b'].append(data_b.ai2.abs().skew())
        fe['ai2_kurt_b'].append(data_b.ai2.kurt())
        fe['ai2_rms_b'].append(np.sqrt(np.mean(data_b.ai2**2)))
        
        fe['circle_ratio_f'].append(data_f[data_f['p']<0.04].shape[0])
        fe['ai1_0_f'].append(data_f[data_f['ai1']>0].shape[0])
        fe['ai1_max_f'].append(data_f.ai1.max())
        fe['ai1_min_f'].append(data_f.ai1.min())
        fe['ai1_mean_f'].append(data_f.ai1.mean())
        fe['ai1_abs_mean_f'].append(data_f.ai1.abs().mean())
        fe['ai1_std_f'].append(data_f.ai1.std())
        fe['ai1_abs_skew_f'].append(data_f.ai1.abs().skew())
        fe['ai1_kurt_f'].append(data_f.ai1.kurt())
        fe['ai1_rms_f'].append(np.sqrt(np.mean(data_f.ai1**2)))
        
        fe['ai2_max_f'].append(data_f.ai2.max())
        fe['ai2_min_f'].append(data_f.ai2.min())
        fe['ai2_mean_f'].append(data_f.ai2.mean())
        fe['ai2_abs_mean_f'].append(data_f.ai2.abs().mean())
        fe['ai2_std_f'].append(data_f.ai2.std())
        fe['ai2_abs_skew_f'].append(data_f.ai2.abs().skew())
        fe['ai2_kurt_f'].append(data_f.ai2.kurt())
        fe['ai2_rms_f'].append(np.sqrt(np.mean(data_f.ai2**2)))
    return pd.DataFrame(fe)
train = get_data(train_ne_file,train_ne_path)
train['result'] = 0
train_po = get_data(train_po_file,train_po_path)
train_po['result'] = 1
train = train.append(train_po).reset_index(drop=True)
save_variable(train,'../data/train.pkl')
test = get_data(test_file_name,test_path)
save_variable(test,'../data/test.pkl')

100%|██████████████████████████████████████████████████████████████████████████████| 2056/2056 [04:19<00:00,  8.36it/s]


'../data/testf.pkl'

In [8]:
train = load_variavle('../data/train.pkl')
test = load_variavle('../data/test.pkl')
print(train.shape, test.shape)

(530, 46) (2056, 45)


# LGB

In [9]:
col = [i for i in train.columns if i not in ['idx', 'result']]
X_train = train[col]
y_train = train['result'].astype(int)
X_test = test[col]
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train),columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)

In [10]:
K = 5
seed = 2018
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
lgb_params = {
                        'boosting_type': 'gbdt',
                        'objective': 'binary',
                        'metric': 'auc',
                        'is_unbalance': True,
                        'max_depth':6,
                        'bagging_freq': 2,
                        'bagging_seed':11,
                        'lambda_l2': 0.2,
                        'subsample': 0.8,
                        'colsample_bytree': 0.6,
                        'learning_rate': 0.1,
                        'seed': 2017,
                        'nthread': 6,
                        'verbose':-1,
             }

In [11]:
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
feature_importance_df = pd.DataFrame()
score = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    lgb_train = lgb.Dataset(X_tr,y_tr)
    lgb_val = lgb.Dataset(X_val,y_val)
    num_round = 100
    clf = lgb.train(lgb_params, lgb_train, num_round, valid_sets = [lgb_train, lgb_val],
                    verbose_eval=-1, early_stopping_rounds = 10
                   )
    oof[val_index] = clf.predict(X_val, num_iteration=clf.best_iteration)
    r_score = lgb_recall_score(y_val.values, clf.predict(X_val, num_iteration=clf.best_iteration),0.3)
    score.append(r_score)
    print('best iteration = ',clf.best_iteration)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = clf.feature_name()
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / skf.n_splits
print('recall_score : ', score)

fold 0
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[22]	training's auc: 0.999583	valid_1's auc: 0.993333
best iteration =  22
fold 1
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[7]	training's auc: 0.999115	valid_1's auc: 1
best iteration =  7
fold 2
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[2]	training's auc: 0.996615	valid_1's auc: 1
best iteration =  2
fold 3
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[8]	training's auc: 0.99875	valid_1's auc: 0.9925
best iteration =  8
fold 4
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[2]	training's auc: 0.991667	valid_1's auc: 0.995
best iteration =  2
recall_score :  [1.0, 1.0, 1.0, 1.0, 1.0]


In [13]:
"""lgb3500:0.63,"""
sub = test[['idx']].copy()
sub['result'] = predictions
print(sub['result'].value_counts().head())
sub = sub.sort_values('result').reset_index(drop=True)
sub.loc[:3500,'result'] = 0
sub.loc[3500:,'result'] = 1
sub['result'] = sub['result'].astype(int)
# sub.to_csv('../sub/lgb3500_fetrainpkl.csv', index=False)

0.264086    888
0.263792    230
0.263546    221
0.263411    215
0.262889    200
Name: result, dtype: int64


In [15]:
# feature_importance_df.groupby('Feature')['importance'].mean().sort_values(ascending=False).reset_index()

# ADAboost

In [25]:
col = ['ai1_kurt_f', 'ai1_max_f', 'ai1_kurt_b', 'ai2_mean_f',
 'ai2_abs_skew_b',  'ai2_kurt_b', 'ai1_mean_b', 'ai1_0_f',
 'ai1_min_f', 'ai2_min_b', 'ai1_abs_skew_b', 'ai2_abs_mean_b', 
 'ai1_kurt_p', 'ai1_mean_f', 'ai1_mean_p', 'ai2_min_f']
X_train = train[col]
y_train = train['result'].astype(int)
X_test = test[col]

In [26]:
K = 5
seed = 10086
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
score = []
prec_score = []
min_p = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    clf = AdaBoostClassifier(n_estimators=150, learning_rate=0.24)
    clf.fit(X_tr,y_tr)
    oof[val_index] = clf.predict_proba(X_val)[:, 1]
    r_score = lgb_recall_score(y_val.values, clf.predict_proba(X_val)[:, 1],0.0045)
    score.append(r_score)
    predictions += clf.predict_proba(X_test)[:, 1]
print('recall_score : ', score)

recall_score :  [1.0, 1.0, 1.0, 1.0, 1.0]


In [20]:
sub = test[['idx']].copy()
sub['result'] = predictions/5
print(sub['result'].value_counts().head())
sub = sub.sort_values('result').reset_index(drop=True)
# sub.loc[:4200,'result'] = 0
# sub.loc[4200:,'result'] = 1
# sub['result'] = sub['result'].astype(int)
# sub.to_csv('../sub/ada4200_fetrainpkl.csv', index=False)

0.001129    15
0.000773    11
0.009707     9
0.007329     9
0.006105     8
Name: result, dtype: int64


# RandomForestClassifier

In [21]:
col = ['ai1_kurt_f', 'ai1_max_f', 'ai1_kurt_b', 'ai2_mean_f',
 'ai2_abs_skew_b',  'ai2_kurt_b', 'ai1_mean_b', 'ai1_0_f',
 'ai1_min_f', 'ai2_min_b', 'ai1_abs_skew_b', 'ai2_abs_mean_b', 
 'ai1_kurt_p', 'ai1_mean_f', 'ai1_mean_p', 'ai2_min_f']
X_train = train[col]
y_train = train['result']
X_test = test[col]

In [22]:
K = 5
seed = 195
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)

In [24]:
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
score = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    """4,126"""
    clf = RandomForestClassifier(max_depth=4,n_estimators=70,random_state=3)
    clf.fit(X_tr,y_tr)
    oof[val_index] = clf.predict_proba(X_val)[:, 1]
    r_score = lgb_recall_score(y_val.values, clf.predict_proba(X_val)[:, 1],0.0045)
    score.append(r_score)
    predictions += clf.predict_proba(X_test)[:, 1]
print('recall_score : ', score)

fold 0
fold 1
fold 2
fold 3
fold 4
recall_score :  [1.0, 1.0, 1.0, 1.0, 1.0]


In [26]:
sub = test[['idx']].copy()
sub['result'] = predictions/5
print(sub['result'].value_counts().head())
sub = sub.sort_values('result').reset_index(drop=True)
sub.loc[:2800,'result'] = 0
sub.loc[2800:,'result'] = 1
sub['result'] = sub['result'].astype(int)
sub.to_csv('../sub/rf2800_fe_mybaseline.csv', index=False)

0.000156    289
0.000085    257
0.000545    171
0.000180    166
0.000110    157
Name: result, dtype: int64


# GradientBoostingClassifier

In [21]:
col = ['ai1_kurt_f', 'ai1_max_f', 'ai1_kurt_b', 'ai2_mean_f',
 'ai2_abs_skew_b',  'ai2_kurt_b', 'ai1_mean_b', 'ai1_0_f',
 'ai1_min_f', 'ai2_min_b', 'ai1_abs_skew_b', 'ai2_abs_mean_b', 
 'ai1_kurt_p', 'ai1_mean_f', 'ai1_mean_p', 'ai2_min_f']
X_train = train[col]
y_train = train['result']
X_test = test[col]

K = 5
seed = 985
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)

"""归一化"""
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
score = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    clf = GradientBoostingClassifier(n_estimators=65, subsample = 0.86, learning_rate=0.105,
                                     max_features='log2', random_state=6)
    clf.fit(X_tr,y_tr)
    oof[val_index] = clf.predict_proba(X_val)[:, 1]
    r_score = lgb_recall_score(y_val.values, clf.predict_proba(X_val)[:, 1],0.0045)
    score.append(r_score)
    predictions += clf.predict_proba(X_test)[:, 1]
print('recall_score : ', score)

fold 0
fold 1
fold 2
fold 3
fold 4
recall_score :  [1.0, 1.0, 1.0, 1.0, 1.0]


In [29]:
sub = test[['idx']].copy()
sub['result'] = predictions/5
print(sub['result'].value_counts().head())
sub = sub.sort_values('result').reset_index(drop=True)
sub.loc[:4000,'result'] = 0
sub.loc[4000:,'result'] = 1
sub['result'] = sub['result'].astype(int)
sub.to_csv('../sub/gbdt4000_fe_my_baseline.csv', index=False)

0.004793    160
0.004755    128
0.004739    109
0.004701     97
0.004790     78
Name: result, dtype: int64


# ExtraTreesClassifier

In [30]:
col = ['ai1_kurt_f', 'ai1_max_f', 'ai1_kurt_b', 'ai2_mean_f',
 'ai2_abs_skew_b',  'ai2_kurt_b', 'ai1_mean_b', 'ai1_0_f',
 'ai1_min_f', 'ai2_min_b', 'ai1_abs_skew_b', 'ai2_abs_mean_b', 
 'ai1_kurt_p', 'ai1_mean_f', 'ai1_mean_p', 'ai2_min_f']
X_train = train[col]
y_train = train['result']
X_test = test[col]

K = 5
seed = 500
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)

"""归一化"""
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
score = []
for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    clf = ExtraTreesClassifier(n_estimators=66,max_depth=1,random_state=101)
    clf.fit(X_tr,y_tr)
    oof[val_index] = clf.predict_proba(X_val)[:, 1]
    r_score = lgb_recall_score(y_val.values, clf.predict_proba(X_val)[:, 1],0.0045)
    score.append(r_score)
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = col
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    predictions += clf.predict_proba(X_test)[:, 1]
print('recall_score : ', score)

fold 0
fold 1
fold 2
fold 3
fold 4
recall_score :  [1.0, 1.0, 1.0, 1.0, 1.0]


In [32]:
sub = test[['idx']].copy()
sub['result'] = predictions/5
print(sub['result'].value_counts().head())
sub = sub.sort_values('result').reset_index(drop=True)
sub.loc[:1200,'result'] = 0
sub.loc[1200:,'result'] = 1
sub['result'] = sub['result'].astype(int)
sub.to_csv('../sub/ext1200_fe_my_baseline.csv', index=False)

0.050492    24
0.043109    22
0.058298    20
0.058481    19
0.041893    15
Name: result, dtype: int64
